In [3]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#%matplotlib inline

In [4]:
df_group =  pd.DataFrame(
    {
        "A": ["foo", "bar", "foo", "bar", "foo", "bar", "foo", "foo"],
        "B": ["one", "one", "two", "three", "two", "two", "one", "three"],
        "C": np.random.randn(8),
        "D": np.random.randn(8),
    }
)
df_group

,A,B,C,D
0,foo,one,2.229890,0.960097
1,bar,one,0.278760,-1.856104
2,foo,two,-1.255221,0.914216
3,bar,three,-0.321690,-0.315118
4,foo,two,-0.011815,0.241089
5,bar,two,-0.963228,-2.166012
6,foo,one,0.153662,0.328204
7,foo,three,-0.067127,-0.534788


In [7]:
df_group.sort_values("C")


,A,B,C,D
2,foo,two,-1.255221,0.914216
5,bar,two,-0.963228,-2.166012
3,bar,three,-0.321690,-0.315118
7,foo,three,-0.067127,-0.534788
4,foo,two,-0.011815,0.241089
6,foo,one,0.153662,0.328204
1,bar,one,0.278760,-1.856104
0,foo,one,2.229890,0.960097


# GroupBy

In [8]:
df_group.groupby("A").count()

,B,C,D
A,,,
bar,3,3,3
foo,5,5,5


In [45]:
df_group.groupby("A","B").count()

C  D
A   B          
bar one    1  1
    three  1  1
    two    1  1
foo one    2  2
    three  1  1
    two    2  2

In [46]:
df_group.groupby(["A"]).groups

{'bar': [1, 3, 5], 'foo': [0, 2, 4, 6, 7]}

In [56]:
df_group.groupby(["A","B"]).size()

A    B    
bar  one      1
     three    1
     two      1
foo  one      2
     three    1
     two      2
dtype: int64

In [49]:
df_group.groupby(["A"],as_index=False).count()

,A,B,C,D
0,bar,3,3,3
1,foo,5,5,5


In [58]:
list_na= [[1, 2, 3], [1, None, 4], [2, 1, 3], [1, 2, 2]]

In [59]:
df_list_na=pd.DataFrame(list_na,columns=["a", "b", "c"])

In [60]:
df_list_na

,a,b,c
0,1,2.0,3
1,1,NaN,4
2,2,1.0,3
3,1,2.0,2


In [65]:
df_list_na.groupby(["a"]).agg([np.sum,np.mean]).rename(columns={"sum":"SUM","mean":"MEAN"})

b        c     
   SUM MEAN SUM MEAN
a                   
1  4.0  2.0   9  3.0
2  1.0  1.0   3  3.0

In [66]:
df_list_na.groupby(["a"]).agg(["sum","mean"]).rename(columns={"sum":"SUM","mean":"MEAN"})

b        c     
   SUM MEAN SUM MEAN
a                   
1  4.0  2.0   9  3.0
2  1.0  1.0   3  3.0

In [120]:
df = pd.DataFrame({"a": [1, 0, 0], "b": [0, 1, 0], "c": [1, 0, 0], "d": [2, 3, 4]})
df_copy = df.copy()
df_copy.loc["sum"]=df.sum() # Add df.sum series as the last row of original df
df_copy

,a,b,c,d
0,1,0,1,2
1,0,1,0,3
2,0,0,0,4
sum,1,1,1,9


In [128]:
#df = pd.concat([df,df.sum().to_frame().T])
df_copy = df.copy()
df_copy_with_sum = pd.concat([df_copy,df_copy.sum().to_frame(name="sum").T]) # Same solution as previous

In [140]:
df_copy_with_sum

,a,b,c,d
0,1,0,1,2
1,0,1,0,3
2,0,0,0,4
sum,1,1,1,9


In [172]:
df.sum()

a    1
b    1
c    1
d    9
dtype: int64

In [170]:
#https://pandas.pydata.org/docs/user_guide/groupby.html#regrouping-by-factor
df.groupby(df.sum(), axis=1).sum() # Group by function:df.sum 

,1,9
0,2,2
1,1,3
2,0,4


In [167]:
dfg = pd.DataFrame({"A": [1, 1, 2, 3, 2], "B": list("aaaba")})
dfg.groupby(["A"]).groups

{1: [0, 1], 2: [2, 4], 3: [3]}

# Transformation


## apply function

In [7]:
df = pd.DataFrame({'A': 'a a b'.split(),
                   'B': [1,2,3],
                   'C': [4,6,5]})
g1 = df.groupby('A', group_keys=False)
g2 = df.groupby('A', group_keys=True)
g1[['B', 'C']].apply(lambda x: x / x.sum())


,B,C
0,0.333333,0.4
1,0.666667,0.6
2,1.000000,1.0


In [10]:
g2[['B', 'C']].apply(lambda x: x / x.sum())

B    C
A                 
a 0  0.333333  0.4
  1  0.666667  0.6
b 2  1.000000  1.0

## transform function

In [8]:
g1[['B', 'C']].transform(lambda x: x / x.sum())


,B,C
0,0.333333,0.4
1,0.666667,0.6
2,1.000000,1.0


In [11]:
g2[['B', 'C']].transform(lambda x: x / x.sum())


,B,C
0,0.333333,0.4
1,0.666667,0.6
2,1.000000,1.0


In [67]:
index = pd.date_range("10/1/1999", periods=1100)
ts = pd.Series(np.random.normal(0.5, 2, 1100), index)


In [88]:
ts.shape
grouped = ts.groupby(lambda x: x.year)

In [85]:
ts = ts.rolling(window=100, min_periods=100).mean().dropna()
ts.head()
ts.shape

(803,)

In [84]:
transformed = ts.groupby(lambda x: x.year).transform(
    lambda x: (x - x.mean()) / x.std()
)
transformed.shape

(902,)

## filter function


In [13]:
dff = pd.DataFrame({"A": np.arange(8), "B": list("aabbbbcc")})
dff

,A,B
0,0,a
1,1,a
2,2,b
3,3,b
4,4,b
5,5,b
6,6,c
7,7,c


In [14]:
dff.groupby("B").filter(lambda x: len(x) > 2)


,A,B
2,2,b
3,3,b
4,4,b
5,5,b


In [15]:
dff.groupby("B").apply(lambda x: len(x))

B
a    2
b    4
c    2
dtype: int64

In [16]:
dff.groupby("B").filter(lambda x: len(x) > 2, dropna=False)

,A,B
0,NaN,NaN
1,NaN,NaN
2,2.0,b
3,3.0,b
4,4.0,b
5,5.0,b
6,NaN,NaN
7,NaN,NaN


In [15]:
dff["C"] = np.arange(8) # Add a new column
dff.groupby("B").filter(lambda x: len(x["C"]) > 2)


,A,B,C
2,2,b,2
3,3,b,3
4,4,b,4
5,5,b,5


## Windowing

In [9]:
s = pd.Series(range(10))
windows = s.rolling(window=5,center=True)
windows.sum()
#windows

0     NaN
1     NaN
2    10.0
3    15.0
4    20.0
5    25.0
6    30.0
7    35.0
8     NaN
9     NaN
dtype: float64